In [ ]:
#Conda 환경 설정
!conda create -n convert
!conda activate convert
!conda install jupyter notebook
!python3 -m ipykernel install --user --name convert --display-name "convert"

In [ ]:
#update
!sudo apt update --yes
!sudo apt upgrade --yes
#you make wanna run below command to remove unwanted
#and free up some space
!sudo apt autoremove --yes

In [ ]:
#lib 설치
!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
!conda install -c conda-forge onnx

In [ ]:
%cd /home/ai/Desktop/work/ai_boostcamp/final/pth_convert

Pytorch를 ONNX로 변환

In [ ]:
import torch
import onnx
#https://github.com/pytorch/vision/blob/main/torchvision/models/segmentation/deeplabv3.py
#deeplabv3_mobilenet_v3_large_coco": "https://download.pytorch.org/models/deeplabv3_mobilenet_v3_large-fc3c493d.pth
torch_model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_mobilenet_v3_large', pretrained=True)
#model_path = './deeplabv3_mobilenet_v3_large-fc3c493d.pth'
model_path = './model_weights.pth'
torch_model.load_state_dict(torch.load(model_path))
torch_model.eval()

batch_size = 1
#[batch_size, channels, height, width]
x = torch.randn(batch_size, 3, 480, 640, requires_grad=True)
torch_out = torch_model(x)

# 모델 변환
torch.onnx.export(torch_model,               # 실행될 모델
                  x,                         # 모델 입력값 (튜플 또는 여러 입력값들도 가능)
                  "onnx.onnx",   # 모델 저장 경로 (파일 또는 파일과 유사한 객체 모두 가능)
                  #export_params=True,        # 모델 파일 안에 학습된 모델 가중치를 저장할지의 여부 default True
                  opset_version=10,          # 모델을 변환할 때 사용할 ONNX 버전
                  #do_constant_folding=False,  # 최적하시 상수폴딩을 사용할지의 여부 Default False
                  input_names = ['input'],   # 모델의 입력값을 가리키는 이름
                  output_names = ['output'] # 모델의 출력값을 가리키는 이름
                  )
onnx_model = onnx.load("onnx.onnx")
onnx.checker.check_model(onnx_model)

ONNX를 TensorFlow로 변환

In [ ]:
!pip install onnx_tf
!sudo apt update --yes
!sudo apt install python3-dev python3-pip python3-venv --yes
!pip install --upgrade tensorflow

In [ ]:
from onnx_tf.backend import prepare
import onnx

TF_PATH = "./pb.pb" # where the representation of tensorflow model will be stored
ONNX_PATH = "./onnx.onnx" # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model

# prepare function converts an ONNX model to an internel representation
# of the computational graph called TensorflowRep and returns
# the converted representation.
tf_rep = prepare(onnx_model)  # creating TensorflowRep object

# export_graph function obtains the graph proto corresponding to the ONNX
# model associated with the backend representation and serializes
# to a protobuf file.
tf_rep.export_graph(TF_PATH)

TensorFlow를 TensorFlow Lite로 변환

In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("./pb.pb") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('tflite.tflite', 'wb') as f:
  f.write(tflite_model)